In [1]:
import time
from queue import Empty as QueueEmpty
import heapq

In [ ]:
import time
from queue import Empty as QueueEmpty
import heapq

def transactionEngine(stockId, queue, dbQueue, internalTransactionQueue, logQueue, users, shutdownEvent):
    def marketTransaction(request):
        # Returns a list of internal transactions/ db transactions and user tranasactions
        nonlocal transactions, stockId
        timeStamp = time.time()
        side = request.get("side")
        internalTransactions = []   # Transactions from one user to another
        dbTransactions = [] # Database Transactions
        userTransactions = []   # User Transactions
        if side == "buy":
            buyerTransaction = request
            numberOfStocksRequired = buyerTransaction.get("quantity")
            totalStockBrought = 0
            buyerId = buyerTransaction.get("uId"); buyerTransactionId = buyerTransaction.get("tId")
            priceToBePaidByBuyer = 0
            
            while numberOfStocksRequired > 0 and len(transactions["sell"]) > 0:
                priceOfBestSell, thisTimeStamp, bestSell = heapq.heappop(transactions["sell"]) # 
                numberOfStocksAvailableInBestSell = bestSell.get("quantity")

                stocksInThisTransaction = min(numberOfStocksAvailableInBestSell, numberOfStocksRequired)
                numberOfStocksAvailableInBestSell -= stocksInThisTransaction
                numberOfStocksRequired -= stocksInThisTransaction
                totalStockBrought += stocksInThisTransaction

                priceInThisTransaction = stocksInThisTransaction * priceOfBestSell
                priceToBePaidByBuyer += priceInThisTransaction

                transactions["marketPrice"] = priceOfBestSell

                sellerId = bestSell.get("uId")
                sellerTid = bestSell.get("tId")
                internalTransactionRequest = {
                    "stockId": stockId,
                    "sellerId": sellerId,
                    "sellerTid": sellerTid,
                    "buyerId": buyerId,
                    "buyerTid": buyerTransactionId,
                    "noOfStocks": stocksInThisTransaction,
                    "amount": priceInThisTransaction,
                    "timeStamp": timeStamp
                }

                dbTransactionRequest = {
                    "tId": sellerTid,
                    "uId": sellerId,
                    "stockId": stockId,
                    "side": "sell",
                    "orderType": "market",
                    "quantity": stocksInThisTransaction,
                    "pricePerUnit": priceOfBestSell,
                    "status": "PARTIAL" if numberOfStocksAvailableInBestSell > 0 else "COMPLETED",
                    "timeStamp": timeStamp
                }

                userTransactionRequest = [
                        {
                            "action": "add",
                            "resource": "money",
                            "uId": sellerId,
                            "quantity": priceInThisTransaction
                        }
                    ]

                internalTransactions.append(internalTransactionRequest)
                dbTransactions.append(dbTransactionRequest)
                userTransactions.append(userTransactionRequest)

                if numberOfStocksAvailableInBestSell > 0:
                    # If there are more stocks to sell than the required, add them back to the heap
                    bestSell["quantity"] = numberOfStocksAvailableInBestSell
                    heapq.heappush(transactions["sell"], (priceOfBestSell, thisTimeStamp, bestSell))
            
            # If no stocks were brought then add the empty request
            if totalStockBrought == 0:
                dbTransactionRequest = {
                    "tId": buyerTransactionId,
                    "uId": buyerId,
                    "stockId": stockId,
                    "side": "buy",
                    "orderType": "market",
                    "quantity": totalStockBrought,
                    "pricePerUnit": 0,
                    "status": "IN-COMPLETE",
                    "timeStamp": timeStamp
                }
                return [], [dbTransactionRequest], []
            
            # Now either there are no more stocks left to buy or the request is satisfied
            internalTransactionRequest = {
                "stockId": stockId,
                "buyerId": buyerId,
                "buyerTid": buyerTransactionId,
                "noOfStocks": totalStockBrought,
                "amount": priceToBePaidByBuyer,
                "timeStamp": timeStamp
            }

            dbTransactionRequest = {
                "tId": buyerTransactionId,
                "uId": buyerId,
                "stockId": stockId,
                "side": "buy",
                "orderType": "market",
                "quantity": totalStockBrought,
                "pricePerUnit": priceToBePaidByBuyer / totalStockBrought if totalStockBrought else 0,
                "status": "PARTIAL" if numberOfStocksRequired > 0 else "COMPLETED",
                "timeStamp": timeStamp
            }

            userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "stock",
                    "uId": buyerId,
                    "stockId": stockId,
                    "quantity": totalStockBrought
                }
            ]

            internalTransactions.append(internalTransactionRequest)
            dbTransactions.append(dbTransactionRequest)
            userTransactions.append(userTransactionRequest)

            return internalTransactions, dbTransactions, userTransactions

        else:
            # This is Sell Part
            sellerTransaction = request
            sellerId, sellerTransactionId = sellerTransaction.get("uId"), sellerTransaction.get("tId")
            totalStockSold = 0; totalAmountRecieved = 0
            numberOfStockAvailable = sellerTransaction.get("quantity")
            while numberOfStockAvailable > 0 and len(transactions["buy"]) > 0: # 
                buyPrice, thisTimeStamp, bestBuy = heapq.heappop(transactions["buy"])
                buyPrice *= -1  # Since the data is saved in negative prices for max-heap
                numberOfBestStocksToBuy = bestBuy.get("quantity")

                stocksSoldInThisTransaction = min(numberOfStockAvailable, numberOfBestStocksToBuy)
                priceInThisTransaction = buyPrice * stocksSoldInThisTransaction
                totalAmountRecieved += priceInThisTransaction

                transactions["marketPrice"] = buyPrice
                numberOfStockAvailable -= stocksSoldInThisTransaction
                numberOfBestStocksToBuy -= stocksSoldInThisTransaction
                totalStockSold += stocksSoldInThisTransaction

                buyerId = bestBuy.get("uId"); buyerTid = bestBuy.get("tId")
                internalTransactionRequest = {
                    "stockId": stockId,
                    "sellerId": sellerId,
                    "sellerTid": sellerTransactionId,
                    "buyerId": buyerId,
                    "buyerTid": buyerTid,
                    "noOfStocks": stocksSoldInThisTransaction,
                    "amount": priceInThisTransaction,
                    "timeStamp": timeStamp
                }

                dbTransactionRequest = {
                    "tId": buyerTid,
                    "uId": buyerId,
                    "stockId": stockId,
                    "side": "buy",
                    "orderType": "market",
                    "quantity": stocksSoldInThisTransaction,
                    "pricePerUnit": buyPrice,
                    "status": "PARTIAL" if numberOfBestStocksToBuy > 0 else "COMPLETE",
                    "timeStamp": timeStamp
                }

                userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "stock",
                    "uId": buyerId,
                    "stockId": stockId,
                    "quantity": stocksSoldInThisTransaction
                }
                ]

                if numberOfBestStocksToBuy > 0:
                    bestBuy["quantity"] = numberOfBestStocksToBuy
                    heapq.heappush(transactions["buy"], (-1 * buyPrice, thisTimeStamp, bestBuy))
            
            if totalStockSold == 0:
                dbTransactionRequest = {
                    "tId": sellerTransactionId,
                    "uId": sellerId,
                    "stockId": stockId,
                    "side": "sell",
                    "orderType": "market",
                    "quantity": totalStockSold,
                    "pricePerUnit": 0,
                    "status": "IN-COMPLETE",
                    "timeStamp": timeStamp
                }
                return [], [dbTransactionRequest], []

            
            internalTransactionRequest = {
                "stockId": stockId,
                "sellerId": sellerId,
                "sellerTid": sellerTransactionId,
                "noOfStocks": totalStockSold,
                "amount": totalAmountRecieved,
                "timeStamp": timeStamp
            }

            dbTransactionRequest = {
                "tId": sellerTransactionId,
                "uId": sellerId,
                "stockId": stockId,
                "side": "sell",
                "orderType": "market",
                "quantity": totalStockSold,
                "pricePerUnit": totalAmountRecieved / totalStockSold if totalStockSold else 0,
                "status": "PARTIAL" if numberOfStockAvailable > 0 else "COMPLETED",
                "timeStamp": timeStamp
            }

            userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "money",
                    "uId": sellerId,
                    "quantity": totalAmountRecieved
                }
            ]

            internalTransactions.append(internalTransactionRequest)
            dbTransactions.append(dbTransactionRequest)
            userTransactions.append(userTransactionRequest)

        return internalTransactions, dbTransactions, userTransactions
                
    def limitTransaction(request):
        nonlocal transactions, stockId
        timeStamp = time.time()
        side = request.get("side")
        internalTransactions = []   # Transactions from one user to another
        dbTransactions = [] # Database Transactions
        userTransactions = []   # User Transactions

        if side == "buy":
            buyerTransaction = request
            numberOfStocksRequired = buyerTransaction.get("quantity")
            totalStockBrought = 0
            buyerId = buyerTransaction.get("uId"); buyerTransactionId = buyerTransaction.get("tId")
            maxBuyPrice = buyerTransaction.get("pricePerUnit")
            priceToBePaidByBuyer = 0
            
            while numberOfStocksRequired > 0 and len(transactions["sell"]) > 0 and transactions["sell"][0][0] <= maxBuyPrice:
                priceOfBestSell, thisTimeStamp, bestSell = heapq.heappop(transactions["sell"]) # 
                numberOfStocksAvailableInBestSell = bestSell.get("quantity")

                stocksInThisTransaction = min(numberOfStocksAvailableInBestSell, numberOfStocksRequired)
                numberOfStocksAvailableInBestSell -= stocksInThisTransaction
                numberOfStocksRequired -= stocksInThisTransaction
                totalStockBrought += stocksInThisTransaction

                priceInThisTransaction = stocksInThisTransaction * priceOfBestSell
                priceToBePaidByBuyer += priceInThisTransaction

                transactions["marketPrice"] = priceOfBestSell

                sellerId = bestSell.get("uId")
                sellerTid = bestSell.get("tId")
                internalTransactionRequest = {
                    "stockId": stockId,
                    "sellerId": sellerId,
                    "sellerTid": sellerTid,
                    "buyerId": buyerId,
                    "buyerTid": buyerTransactionId,
                    "noOfStocks": stocksInThisTransaction,
                    "amount": priceInThisTransaction,
                    "timeStamp": timeStamp
                }

                dbTransactionRequest = {
                    "tId": sellerTid,
                    "uId": sellerId,
                    "stockId": stockId,
                    "side": "sell",
                    "orderType": "limit",
                    "quantity": stocksInThisTransaction,
                    "pricePerUnit": priceOfBestSell,
                    "status": "PARTIAL" if numberOfStocksAvailableInBestSell > 0 else "COMPLETED",
                    "timeStamp": timeStamp
                }

                userTransactionRequest = [
                        {
                            "action": "add",
                            "resource": "money",
                            "uId": sellerId,
                            "quantity": priceInThisTransaction
                        }
                    ]

                internalTransactions.append(internalTransactionRequest)
                dbTransactions.append(dbTransactionRequest)
                userTransactions.append(userTransactionRequest)

                if numberOfStocksAvailableInBestSell > 0:
                    # If there are more stocks to sell than the required, add them back to the heap
                    bestSell["quantity"] = numberOfStocksAvailableInBestSell
                    heapq.heappush(transactions["sell"], (priceOfBestSell, thisTimeStamp, bestSell))
            
            # If no stocks were brought then add the empty request
            if totalStockBrought == 0:
                heapq.heappush(transactions["buy"], (-request.get("pricePerUnit"), request.get("timeStamp"), request))
                return [], [], []
            
            if numberOfStocksRequired > 0:
                request["quantity"] = numberOfStocksRequired
                heapq.heappush(transactions["buy"], (-request.get("pricePerUnit"), request.get("timeStamp"), request))
            
            internalTransactionRequest = {
                "stockId": stockId,
                "buyerId": buyerId,
                "buyerTid": buyerTransactionId,
                "noOfStocks": totalStockBrought,
                "amount": priceToBePaidByBuyer,
                "timeStamp": timeStamp
            }

            dbTransactionRequest = {
                "tId": buyerTransactionId,
                "uId": buyerId,
                "stockId": stockId,
                "side": "buy",
                "orderType": "limit",
                "quantity": totalStockBrought,
                "pricePerUnit": priceToBePaidByBuyer / totalStockBrought if totalStockBrought else 0,
                "status": "PARTIAL" if numberOfStocksRequired > 0 else "COMPLETED",
                "timeStamp": timeStamp
            }

            userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "stock",
                    "uId": buyerId,
                    "stockId": stockId,
                    "quantity": totalStockBrought
                }
            ]

            internalTransactions.append(internalTransactionRequest)
            dbTransactions.append(dbTransactionRequest)
            userTransactions.append(userTransactionRequest)

        else:
            # This is Sell Part
            sellerTransaction = request
            sellerId, sellerTransactionId = sellerTransaction.get("uId"), sellerTransaction.get("tId")
            minimumSellingPrice = sellerTransaction.get("pricePerUnit")
            totalStockSold = 0; totalAmountRecieved = 0
            numberOfStockAvailable = sellerTransaction.get("quantity")
            while numberOfStockAvailable > 0 and len(transactions["buy"]) > 0 and -transactions["buy"][0][0] >= minimumSellingPrice:
                buyPrice, thisTimeStamp, bestBuy = heapq.heappop(transactions["buy"])
                buyPrice *= -1  # Since the data is saved in negative prices for max-heap
                numberOfBestStocksToBuy = bestBuy.get("quantity")

                stocksSoldInThisTransaction = min(numberOfStockAvailable, numberOfBestStocksToBuy)
                priceInThisTransaction = buyPrice * stocksSoldInThisTransaction
                totalAmountRecieved += priceInThisTransaction

                transactions["marketPrice"] = buyPrice
                numberOfStockAvailable -= stocksSoldInThisTransaction
                numberOfBestStocksToBuy -= stocksSoldInThisTransaction
                totalStockSold += stocksSoldInThisTransaction

                buyerId = bestBuy.get("uId"); buyerTid = bestBuy.get("tId")
                internalTransactionRequest = {
                    "stockId": stockId,
                    "sellerId": sellerId,
                    "sellerTid": sellerTransactionId,
                    "buyerId": buyerId,
                    "buyerTid": buyerTid,
                    "noOfStocks": stocksSoldInThisTransaction,
                    "amount": priceInThisTransaction,
                    "timeStamp": timeStamp
                }

                dbTransactionRequest = {
                    "tId": buyerTid,
                    "uId": buyerId,
                    "stockId": stockId,
                    "side": "buy",
                    "orderType": "limit",
                    "quantity": stocksSoldInThisTransaction,
                    "pricePerUnit": buyPrice,
                    "status": "PARTIAL" if numberOfBestStocksToBuy > 0 else "COMPLETED",
                    "timeStamp": timeStamp
                }

                userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "stock",
                    "uId": buyerId,
                    "stockId": stockId,
                    "quantity": stocksSoldInThisTransaction
                }
                ]

                if numberOfBestStocksToBuy > 0:
                    bestBuy["quantity"] = numberOfBestStocksToBuy
                    heapq.heappush(transactions["buy"], (-1 * buyPrice, thisTimeStamp, bestBuy))
            
            if totalStockSold == 0:
                heapq.heappush(transactions["sell"], (request.get("pricePerUnit"), request.get("timeStamp"), request))
                return [], [], []

            if numberOfStockAvailable > 0:
                request["quantity"] = numberOfStockAvailable
                heapq.heappush(transactions["sell"], (request.get("pricePerUnit"), request.get("timeStamp"), request))

            internalTransactionRequest = {
                "stockId": stockId,
                "sellerId": sellerId,
                "sellerTid": sellerTransactionId,
                "noOfStocks": totalStockSold,
                "amount": totalAmountRecieved,
                "timeStamp": timeStamp
            }

            dbTransactionRequest = {
                "tId": sellerTransactionId,
                "uId": sellerId,
                "stockId": stockId,
                "side": "sell",
                "orderType": "limit",
                "quantity": totalStockSold,
                "pricePerUnit": totalAmountRecieved / totalStockSold if totalStockSold else 0,
                "status": "PARTIAL" if numberOfStockAvailable > 0 else "COMPLETED",
                "timeStamp": timeStamp
            }

            userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "money",
                    "uId": sellerId,
                    "quantity": totalAmountRecieved
                }
            ]

            internalTransactions.append(internalTransactionRequest)
            dbTransactions.append(dbTransactionRequest)
            userTransactions.append(userTransactionRequest)

        return internalTransactions, dbTransactions, userTransactions
        

    try:
        transactions = {"buy":[], "sell": [], "marketPrice": 0.0}
        while True:
            if shutdownEvent.is_set():
                break
            request = None
            try:
                request = queue.get(timeout=0.01)
            except QueueEmpty as qe:
                print("No Transaction recieved, Lets wait!")
                pass
            
            if request is None:
                continue
            orderType = request.get("orderType")
            if orderType == "limit":
                pass
            elif orderType == "market":
                internalTxns, dbTxns, userTxns = marketTransaction(request)
            elif orderType == "ioc":
                pass
            elif orderType == "pok":
                pass

    except Exception as _e:
        logQueue.put(f"Exception at transaction-engine {stockId}: {str(_e)}")
    
    finally:
        print("Exiting")


In [1]:
import time
st = time.time()
from tinydb import TinyDB

# Open (or create) the database
db = TinyDB('db.json')

# List of transactions (multiple JSON objects like your example)
transactions = [
    {
        "tId": "t12345",
        "uId": "user1",
        "stockId": "btc",
        "side": "buy",
        "orderType": "limit",
        "quantity": 5.0,
        "pricePerUnit": 100.00,
        "status": "RECIEVED",
        "timeStamp": 34234343423
    },
    {
        "tId": "t12346",
        "uId": "user2",
        "stockId": "eth",
        "side": "sell",
        "orderType": "market",
        "quantity": 3.0,
        "pricePerUnit": 200.00,
        "status": "PENDING",
        "timeStamp": 34234343500
    },
    # Add more transactions if you want
]

# Insert multiple transactions at once
db.insert_multiple(transactions)

# Fetch all records
all_transactions = db.all()

# Print all transactions
for tx in all_transactions:
    print(tx)

print(time.time() - st)

{'tId': 't12345', 'uId': 'user1', 'stockId': 'btc', 'side': 'buy', 'orderType': 'limit', 'quantity': 5.0, 'pricePerUnit': 100.0, 'status': 'RECIEVED', 'timeStamp': 34234343423}
{'tId': 't12346', 'uId': 'user2', 'stockId': 'eth', 'side': 'sell', 'orderType': 'market', 'quantity': 3.0, 'pricePerUnit': 200.0, 'status': 'PENDING', 'timeStamp': 34234343500}
0.011000394821166992
